In [429]:
import torch
import random

In [430]:
%run micrograd.ipynb

In [431]:
%run 'neuron backprop'.ipynb

0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w2 0.0


Since we're pretty much clear about the simplified version of Value class and its operations, I've copied and pasted Andrej Karpathy's Value class with many different methods:

In [432]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()


In [433]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

x = [2.0, 3.0]
n = Neuron(2)
n(x)

Value(data=-0.014906951105695721)

In [434]:
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
        
layer = Layer(2, 3)
layer(x)

[Value(data=0.9930293209061191),
 Value(data=-0.9117194617598063),
 Value(data=-0.9991160666458824)]

In [435]:
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [436]:
mlp = MLP(2, [4, 4, 1])
mlp(x)

Value(data=-0.3048178445501472)

Example dataset:

In [437]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets
ypred = [mlp(x) for x in xs]
ypred

[Value(data=-0.3048178445501472),
 Value(data=-0.17667670970230331),
 Value(data=-0.33735027622841446),
 Value(data=-0.4160243061003608)]

In [438]:
# mean squared error loss
# pairing up output and ground truth ys: (y_out - y_gt)^2
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

Value(data=4.824640339684684)

In [439]:
loss.backward()

In [440]:
mlp.layers[0].neurons[0].w[0].grad
# gradient of a particular weight of a particular neuron of a particular layer is positive (+0.3), its influence on the loss is also positive.
# increasing it will increase the loss
# note: the parameters are reinitialized on each run, so the result of the loss might look different

-0.7231866862331077

In [441]:
# draw_dot(loss)

In [442]:
ypred = [mlp(x) for x in xs]
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

Value(data=4.824640339684684)

In [443]:
ypred = [mlp(x) for x in xs]
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

Value(data=4.824640339684684)

In [444]:
for p in n.parameters():
    p.grad = 0.0

In [445]:
loss.backward()

In [446]:
for p in mlp.parameters():
    p.data += -0.001 * p.grad

In [447]:
ypred

[Value(data=-0.3048178445501472),
 Value(data=-0.17667670970230331),
 Value(data=-0.33735027622841446),
 Value(data=-0.4160243061003608)]

I have the original predicted values, and what I did is I completed the forward pass, backpropagated, and updated the weights.\
So run the forward pass -> set the gradient back to zero (so that we don't add up the gradients) -> loss.backward() -> nudge the weigths in a certain direction to minimize loss.\
Repeat the process multiple times until the minimal loss is achieved. I also updated the step from 0.1 to 0.01 to 0.001 as the loss got lower so that I don't overstep the local minimum.

In [450]:
# With loops:

for k in range(20):

    # forward pass
    ypred = [mlp(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    # backward pass
    for p in mlp.parameters():
        p.grad = 0.0
    loss.backward()

    # update
    for p in mlp.parameters():
        p.data += -0.05 * p.grad

    print(k, loss.data)

0 1.1642184757352596
1 1.6327260616127368
2 1.5979802911404664
3 1.8849368120100622
4 1.232718456801852
5 1.5164501885621249
6 1.4230434454300425
7 1.588452319928603
8 1.288452960454221
9 1.350640771725211
10 1.2266320180028263
11 1.2028651569387827
12 1.0876153740932795
13 0.959662647491746
14 0.777270771967605
15 0.4601503736540231
16 0.170292416192645
17 0.09410244805421031
18 0.08664571240327608
19 0.08089408925723507


In [451]:
ypred

[Value(data=0.9579623825155305),
 Value(data=-0.8722419169855368),
 Value(data=-0.8307540499263738),
 Value(data=0.8151741127936629)]